## Read in data 

In [11]:
import json
import re
import spacy as sp
import pandas as pd

with open('data/all_situations.json') as f:
    situations = json.load(f)

print(f'There are {len(situations)} situations in the dataset')

There are 1574 situations in the dataset


In [4]:
#print the first situation
print(json.dumps(situations[0], indent=4))

{
    "id": 16574,
    "uuid": "7f054592-f78f-4163-998f-9417e3f65726",
    "changed": "2024-02-07T12:06:20+01:00",
    "title": "Ik ga naar de oogarts voor een oogontsteking door herpes",
    "link": "https://www.thuisarts.nl/oogontsteking-door-herpes/ik-ga-naar-oogarts-voor-oogontsteking-door-herpes",
    "revision_date": "2020-08-13",
    "subject": {
        "id": "16572",
        "changed": "2022-06-30T14:12:38+02:00",
        "title": "Oogontsteking door herpes",
        "link": "https://www.thuisarts.nl/oogontsteking-door-herpes"
    },
    "content": [
        {
            "index": 0,
            "type": "text",
            "category": "In het kort",
            "uuid": "4ed6dbef-035d-482b-92ea-a640ec4b268e",
            "item": [
                {
                    "title": "In het kort",
                    "value": "<ul><li>De oogarts onderzoekt uw ogen met kleurstof en blauw licht.</li>\n\t<li>Er zijn verschillende behandelingen:\n\t<ul><li>oogzalf met aciclovir</li>\n\t\

## Exploratory Data Analysis

In [16]:
data1 = {
    'id' : [],
    'situation_title': [],
    'related_subject': [],
    'content': [],
    'icpc': [],
    'medical_source': [],
    'related_situations': []
}

for situation in situations:
    data1['id'].append(situation['id']) #extract the id 
    data1['situation_title'].append(situation['title']) #extract the title of the subject
    #data1['related_subject'].append(situation['subject']) #extract the related subject
    #data1['content'].append(situation['content']) #extract the content of the subject
    icpc_codes = situation['codes']['icpc'] #extract the icpc code of the subject
    if icpc_codes:
        data1['icpc'].append(', '.join(icpc_codes))
    else:
        data1['icpc'].append(None)  # if there is no icpc code, give None
    medical_sources = situation['codes']['medical_source'] #extract the medical source of the subject
    if medical_sources:
        data1['medical_source'].append(', '.join(medical_sources))
    else:
        data1['medical_source'].append(None)  # if there is no medical source, give None
    # related_situations = situation['related']['situations'] #extract the related situations of the subject
    # if related_situations:
    #     for related_situation in related_situations:
    #         data1['related_situations'].append(related_situation['title'])
    # else:
    #     data1['related_situations'].append(None)  # if there is no related situation, give None

df1 = pd.DataFrame(data1)

print(df1.head())


ValueError: All arrays must be of the same length

#### Extract situations with corresponding categories, titles and their texts

In [3]:
# Function to extract and organize information from a situation
def extract_info(situation):
    info = {
        'title': situation['title'],
        'subject': {
            'title': situation['subject']['title'],
            'link': situation['subject']['link']
        },
        'content': []
    }

    for content_item in situation['content']:
        category = content_item['category']
        items = []

        for item in content_item['item']:
            item_info = {'title': item['title'], 'value': item.get('value', None) }
            items.append(item_info)

        info['content'].append({'category': category, 'items': items})

    return info

# Loop through situations and extract information
#situations = data['related']['situations']
organized_data = {}

for situation in data:
    situation_id = situation['id']
    organized_data[situation_id] = extract_info(situation)

# Print the organized data
for situation_id, info in organized_data.items():
    print(f"Situation ID: {situation_id}")
    print(f"Title: {info['title']}")
    print(f"Subject: {info['subject']['title']} ({info['subject']['link']})")

    for content_item in info['content']:
        print(f"\nCategory: {content_item['category']}")
        for item in content_item['items']:
            print(f"{item['title']}: {item['value']}")
    
    print("\n" + "="*50 + "\n")

TypeError: list indices must be integers or slices, not str

In [ ]:
situations = []
for situation in data:
    title = situation['title']
    subject = situation['subject']['title']
    categories = [category['title'] for category in situation['category']]
    items = []
    for item in situation['items']:
        items.append({'title': item['title'], 'value': item['value']})
    situations.append({'title': title, 'subject': subject, 'category': categories, 'items': items})



KeyError: 'category'

In [ ]:
texts_per_cat_per_situation = []

with open('nlp_data.txt', 'w', encoding='utf-8') as file:
    for situation in data:
        title = situation['title']
        file.write(f"Situation: {title}\n\n")
    
        for content_item in situation['content']:
            content_category = content_item['category']
        
            for item in content_item['item']:
                item_title = item['title']
            
                #sometimes there is no 'value' because there is no text, only a video
                item_value = item.get('value', None) 
            
                if item_value is not None:
                    texts_per_cat_per_situation.append(item_value)
                    file.write(f"Category: {content_category}\nCategory Title: {item_title}\nItem Value: {item_value}\n\n")


# separate the text 

#### Number of categories and how many times they occur

In [4]:
def has_wanneer_bellen_category(situation):
    for content_item in situation.get('content', []):
        content_category = content_item.get('category')
        if content_category == 'Wanneer bellen':
            return True
    return False

# Assuming 'data' is a list of situations
categories = {}
missing_categories = []

for situation in data:
    for content_item in situation.get('content', []):
        content_category = content_item.get('category')
        categories[content_category] = categories.get(content_category, 0) + 1

        # Check if "wanneer bellen" category is missing and add the situation to the list
        if content_category == 'Wanneer bellen' and not has_wanneer_bellen_category(situation):
            missing_categories.append(situation)

# Print categories and their counts
print("Categories and their counts:")
for category, count in categories.items():
    print(f"{category}: {count}")

print(f'There are {len(categories)} categories in the data.')

sorted_categories = sorted(categories.items(), key=lambda x: x[1], reverse=True)
print("Sorted categories:", sorted_categories)

print(f'Situations missing "wanneer bellen" category: {len(missing_categories)}')

Categories and their counts:
In het kort: 1573
Geen wondroos krijgen: 1
Wat kan helpen?: 1
Goed voor je huid zorgen: 1
Wondjes verzorgen: 1
Wanneer bellen: 833
Meer informatie: 1535
Over deze tekst: 65
Film Wondroos: 2
Wat is het: 995
Wat merk ik: 462
Oorzaken: 596
Wie heeft grotere kans: 1
Kan het kwaad: 116
Behandeling: 418
Adviezen: 733
Hoe gaat het verder: 990
Adviezen niet weer wondroos: 1
Onderzoeken: 291
Wat kan ik zelf doen: 2
Medicijnen: 325
Film: 168
Besmetting: 4
Prik: 3
Wanneer: 18
Wanneer niet: 4
Voordelen en nadelen: 24
Griepprik: voor wie: 1
Film: Uitnodiging voor de griepprik: 1
Griepprik: wanneer: 1
Griepprik: werking: 1
Griepprik: elk jaar: 1
Griepprik: voordelen en nadelen: 1
Wat is griep: 1
Film: Griep voorkomen: 1
Besmetting voorkomen: 1
Prik tegen pneumokokken: 1
Zalven en crèmes: 1
Behandeling met licht: 1
Pillen: 5
Keuzekaart: behandelingen: 2
Zelf doen: 5
Na het onderzoek: 1
Keuzekaart behandeling eczeem: 1
Keuzehulp: 25
Bijwerkingen: 59
Soorten: 4
Activiteiten

In [7]:
categories = {}
for situation in data:
    for content_item in situation['content']:
        content_category = content_item['category']
        categories[content_category] = categories.get(content_category, 0) + 1

print(f'There are {len(categories)} categories in the data.')

#how many categories have only 1 item?
single_item_categories = []
for category, count in categories.items():
    if count == 1:
        single_item_categories.append(category)

print(f'There are {len(single_item_categories)} categories with only 1 item.')

sorted_categories = sorted(categories.items(), key=lambda x: x[1], reverse=True)
sorted_categories

There are 2345 categories in the data.
There are 1785 categories with only 1 item.


[('In het kort', 1571),
 ('Meer informatie', 1533),
 ('Wat is het', 994),
 ('Hoe gaat het verder', 989),
 ('Wanneer bellen', 830),
 ('Adviezen', 732),
 ('Oorzaken', 595),
 ('Wat merk ik', 463),
 ('Behandeling', 418),
 ('Medicijnen', 325),
 ('Onderzoeken', 290),
 ('Film', 168),
 ('Wat merk je', 126),
 ('Kan het kwaad', 116),
 ("Risico's", 107),
 ('Over deze tekst', 62),
 ('Bijwerkingen', 59),
 ('Voorbereiding', 42),
 ('Keuzekaart', 41),
 ('Operatie', 40),
 ('Controles', 39),
 ('Onderzoek', 36),
 ('Uitvoering', 35),
 ('Andere behandelingen', 30),
 ('Is het gevaarlijk', 27),
 ('Klachten', 26),
 ('Na de operatie', 26),
 ('Waarom', 26),
 ('Nadelen', 25),
 ('Wat is het?', 25),
 ('Voordelen en nadelen', 24),
 ('Behandelingen', 24),
 ('Makers van de tekst', 22),
 ('Keuzehulp', 21),
 ('Oorzaak', 21),
 ('Risicogroepen', 20),
 ('Hulp', 20),
 ('Voor de operatie', 18),
 ('Wanneer', 18),
 ('Behandeling kiezen', 17),
 ('Wat merk ik?', 16),
 ('Voordelen', 15),
 ('Voor wie', 14),
 ('Hoe werkt het', 14)

In [8]:
nulde = []
eerste = []
tweede = []

for situation in data:
    for content_item in situation['content']:
        if content_item['category'] == 'Meer informatie':
            for item in content_item['item']:
                if 'specialist' in item['value']:
                    tweede.append(item['value'])
                elif 'huisarts' in item['value']:
                    eerste.append(item['value'])

print(len(eerste))

753


In [9]:
#classification based on 'Wanneer bellen' category and whether the text contains 'huisarts', 'ziekenhuis' or 'uw arts' 
nuldelijns_lijst = []
eerstelijns_lijst = []
tweedelijns_lijst = []
overige_lijst = []
for situation in data:
    for content_item in situation['content']:
        if content_item['category'] == 'Wanneer bellen':
            for item in content_item['item']:
                if 'huisarts' in item['value']:
                    nuldelijns_lijst.append(situation['title'])
                elif 'ziekenhuis' in item['value']:
                    eerstelijns_lijst.append(situation['title'])
                elif 'uw arts' in item['value']:
                    tweedelijns_lijst.append(situation['title'])
                else:
                    overige_lijst.append(situation['title'])

print(f'There are {len(nuldelijns_lijst)} situations in the nuldelijns_lijst')
print(f'There are {len(eerstelijns_lijst)} situations in the eerstelijns_lijst')
print(f'There are {len(tweedelijns_lijst)} situations in the tweedelijns_lijst')
print(f'There are {len(overige_lijst)} situations in the overige_lijst')

There are 664 situations in the nuldelijns_lijst
There are 40 situations in the eerstelijns_lijst
There are 18 situations in the tweedelijns_lijst
There are 108 situations in the overige_lijst


In [10]:
#show me the situations in the eerstelijns_lijst
eerstelijns_lijst

['Ik heb eczeem door contact met iets en ga naar de huid-arts',
 'Ik krijg chemotherapie bij darmkanker',
 'Ik krijg prikken in de rug bij een hernia',
 'Ik ben tussen de 34 en 37 weken zwanger en mijn vliezen zijn gebroken',
 'Ik heb zaadbalkanker en krijg een operatie om mijn zaadbal weg te halen',
 'Ik heb zaadbalkanker en krijg bestraling',
 'Ik heb zaadbalkanker en krijg chemotherapie',
 'Ik krijg behandeling voor colitis ulcerosa',
 'Ik krijg behandeling voor de ziekte van Crohn',
 'Ik krijg een halve kunstknie',
 'Ik krijg een kunstknie',
 'Ik heb een borstoperatie gehad bij borstkanker. Hoe gaat het verder?',
 'Ik krijg een buikoperatie bij eierstokkanker (stadieringsoperatie)',
 'In mijn familie komt eierstokkanker voor',
 'Ik krijg een buikoperatie bij uitgezaaide eierstokkanker (debulking)',
 'Ik kies voor een maagverkleining',
 'Ik heb longkanker zonder uitzaaiingen (stadium 1 of 2)',
 'Ik heb een melanoom en krijg een 2e operatie',
 'Ik word geopereerd aan de halsslagader'

In [11]:
#make a dictionary with all categories per situation
category_dict = {}
for situation in data:
    for content_item in situation['content']:
        category_dict[situation['title']] = category_dict.get(situation['title'], []) + [content_item['category']]
#print(category_dict)

#if a situation does not contain the category 'Wanneer bellen', add it to the list
missing_wanneer_bellen= []
for situation, categories in category_dict.items():
    if 'Wanneer bellen' not in categories:
        missing_wanneer_bellen.append(situation)

print(missing_wanneer_bellen) #voorbereiden op keuze of voorbereiden op behandeling

['Ik krijg een hersenscan bij onderzoek naar de ziekte van Parkinson', 'Ik ga naar het ziekenhuis voor onderzoek naar de ziekte van Parkinson', 'Ik wil een behandeling kiezen bij de ziekte van Parkinson', 'Ik heb problemen met denken en onthouden door de ziekte van Parkinson', 'Ik heb de ziekte van Parkinson en heb een depressie', 'Ik heb slaapproblemen door de ziekte van Parkinson', 'Ik denk aan euthanasie', 'Ik heb uitzaaiingen van darmkanker en kan misschien genezen', 'Mijn kind heeft eczeem', 'Ik heb eczeem', 'Ik heb blaasjeseczeem', 'Ik heb eczeem op mijn onderbenen', 'Ik heb eczeem door contact met iets', 'Ik heb eczeem door erfelijke aanleg', 'Ik heb artrose in mijn hand en ga naar het ziekenhuis', 'Mijn kind is te zwaar', 'Ik heb de ziekte van Pfeiffer', 'Mijn naaste heeft een verstandelijke beperking en hoort niet goed', 'Mijn naaste met een verstandelijke beperking krijgt onderzoek naar dementie', 'Ik wil mijn naaste met een verstandelijke beperking goed kunnen begrijpen', 'I

## NLP

#### Preprocess text

In [12]:
# Remove HTML tags and escape sequences
html_pattern = re.compile(r'<[^>]+>')
escape_pattern = re.compile(r'\\u[0-9a-fA-F]{4}')

clean_text = [escape_pattern.sub('', html_pattern.sub('', item)) for item in item_values]

texts_per_cat_per_situation = clean_text

NameError: name 'item_values' is not defined

In [13]:
#tokenize, postag and parse text
nlp = sp.load('nl_core_news_sm')
doc = nlp(clean_text[1])
for token in doc:
    print(token.text, token.pos_, token.dep_)

#delete prepositions and stop words, and punctuation and lemmatize tokens (to get canonical form)
lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.pos_ == 'ADP']


#give me all adjacent pairs of words
pairs = []
for i in range(len(lemmas)-1):
    pairs.append((lemmas[i], lemmas[i+1]))
print(pairs)

NameError: name 'clean_text' is not defined